# Ejercicio FASTAPI API
Para este ejercicio tendrás que desplegar un modelo de machine learning en una API para su consumo. Ya tienes un modelo entrenado,desarrollarás una API que permita consumir dicho modelo desde cualquier otra tecnología.

**Se presenta el siguiente caso de uso**

Una empresa distribuidora de ámbito nacional pretende utilizar un modelo desarrollado por el departamento de data science, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web, comunicándose con una BBDD para ingestar o reentrenar el modelo. No vale base de datos en csv. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad. (/v2/predict)
2. Un endpoint para almacenar nuevos registros en la base de datos que deberás crear previamente.(/v2/ingest_data)
3. Posibilidad de reentrenar de nuevo el modelo con los posibles nuevos registros que se recojan. (/v2/retrain)


**NOTAS**: 
1. Deberás desplegarlo desde un repositorio de github.
2. Ojo con la ruta para hacer el load de tu modelo y datos, comprueba cual es la ruta en la que está buscándolo.
3. El desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el desarrollo de una API con FastAPI.

**Entregable**: repositorio github.

In [12]:
!pip install pytest


  Obtaining dependency information for pytest from https://files.pythonhosted.org/packages/c4/43/6b1debd95ecdf001bc46789a933f658da3f9738c65f32db3f4e8f2a4ca97/pytest-8.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for iniconfig from https://files.pythonhosted.org/packages/ef/a6/62565a6e1cf69e10f5727360368e451d4b7f58beeac6173dc9db836a5b46/iniconfig-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for pluggy<2.0,>=1.5 from https://files.pythonhosted.org/packages/88/5f/e351af9a41f866ac3f1fac4ca0613908d9a41741cfcf2228f4ad853b697d/pluggy-1.5.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/339.2 kB ? eta -:--:--
   --- ------------------------------------ 30.7/339.2 kB 1.3 MB/s eta 0:00:01
   ------- ------------------------------- 61.4/339.2 kB 656.4 kB/s eta 0:00:01
   -------------------- ------------------- 174.1/339.2 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------  337.9/339.2 kB 1.9 MB/s eta 0:00:01
   


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import numpy as np
import pandas as pd
import sqlite3


In [5]:
df = pd.read_csv("../data/Advertising.csv", index_col=0)
df

,TV,radio,newpaper,sales
0,230.1,37.8,6s9.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0
...,...,...,...,...
195,38.2,3.7,13.8,7600.0
196,94.2,4.9,8.1,9700.0
197,177.0,9.3,6.4,12800.0
198,283.6,42.0,66.2,25500.0


### Limpiamos la columna de newpaper que tiene una errata

In [6]:
df.rename(columns={"newpaper":"newspaper"}, inplace=True)
df['newspaper'] = df['newspaper'].str.replace("s", "")
df['newspaper'] = df['newspaper'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


### Creamos la conexión a nuetra base de datos, si no existe la creará

In [8]:
conn = sqlite3.connect('Advertising.db')
cur = conn.cursor()

### Creamos nuestra tabla de la base de datos

In [9]:
cur.execute('''CREATE TABLE IF NOT EXISTS Ventas (
                id INTEGER PRIMARY KEY,
                TV ,
                radio ,
                newspaper ,
                sales 
                )''')


### La rellenamos con los datos de nuestro df 

In [11]:
df.to_sql('Ventas', conn, if_exists='replace', index=False)

200